In [ ]:
# default_exp scripting

# Scripting
> Automatically detect hardware and devices like FPGAs and SoCs, read/write to memory, program, compile, archive projects, generate LUT masks, perform timing analyses, and more.

In [ ]:
#hide
from nbdev import *
from nbdev.imports import *
from nbdev.export import *
from nbdev.sync import *
from nbdev.showdoc import *

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import warnings
with warnings.catch_warnings(): #ignore warnings
    warnings.simplefilter("ignore")
    import sidis
    import mif
    import quartustcl
    import numpy as np
    import os
    from typing import Optional, Tuple, Dict, Callable, Union
    import functools
    from functools import wraps

## Quartus Shell and Device Detection

In [ ]:
#export
def qshell(func):
    '''Function decorator that opens quartustcl shell if none open.'''
    @wraps(func)
    def wrap(q=None,*args,**kwargs):
        if q is None:
            with quartustcl.QuartusTcl() as q:
                result = func(q,*args,**kwargs)
        else:
            result = func(q,*args,**kwargs)
        return result
    return wrap

In [ ]:
show_doc(qshell)

<h4 id="qshell" class="doc_header"><code>qshell</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>qshell</code>(**`func`**)

Function decorator that opens quartustcl shell if none open.

For the following I will use two DE-10 Nano SoC FPGAs attached via USB. Note that function wrappers disrupt the automatically generated doc strings created by nbdev. As such, we will show the source code for details.

In [ ]:
#export
@qshell
def get_fpgas(q : Optional[quartustcl.QuartusTcl] = None,
             parse : bool = False):            
    '''
    Get all connected FPGAs and SOCs as lists of hardware names `hwnames` and device names `devnames`.
    Uses `q` shell if given, otherwise opens new shell. If `parse`, extracts only the FPGA, ignoring SoCs.
    Returns a dict keyed by hardware port (e.g USB) and valued with devices (e.g FPGA/SOC).
    '''
    hwnames = q.parse(q.get_hardware_names())
    devnames = [q.parse(q.get_device_names(hardware_name=h)) for h in hwnames]
    if parse:
        devnames=[d[-1] for d in devnames] #SoC is always first, if it exists
    return dict(list(zip(hwnames,devnames)))

In [ ]:
get_fpgas()

{'DE-SoC [USB-1]': ['@1: SOCVHPS (0x4BA00477)',
  '@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)'],
 'DE-SoC [USB-2]': ['@1: SOCVHPS (0x4BA00477)',
  '@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)']}

In [ ]:
get_fpgas(parse=True) #strip the SoC

{'DE-SoC [USB-1]': '@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)',
 'DE-SoC [USB-2]': '@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)'}

In [ ]:
#export
@qshell
def get_insts(q = None,
              N_levels=2,
              hwnames=None,
              devnames=None):
    '''
    Get all memory instances from `hwname` and `devname`. See `QuartusTcl` doc for `N_levels`.
    '''
    if hwnames is None or devnames is None:
        device_dict=get_fpgas(parse=True)
        hwnames=list(device_dict.keys())
        devnames=list(device_dict.values())
    'Finds instance index given a name `inst_name` (string).'
    memories={}
    for hwname,devname in zip(hwnames,devnames):
        memories_raw = q.get_editable_mem_instances(hardware_name=hwname,\
            device_name=devname)
        memories[hwname]=q.parse(memories_raw, levels=N_levels)
    return memories

In [ ]:
get_insts()

{'DE-SoC [USB-1]': [['0', '1', '1', 'RW', 'ROM/RAM', 'BIT'],
  ['1', '1', '3', 'RW', 'ROM/RAM', 'MULT'],
  ['2', '512', '1300', 'RW', 'ROM/RAM', 'READ'],
  ['3', '512', '1300', 'RW', 'ROM/RAM', 'READ'],
  ['4', '512', '1300', 'RW', 'ROM/RAM', 'READ']],
 'DE-SoC [USB-2]': [['0', '1', '1', 'RW', 'ROM/RAM', 'BIT'],
  ['1', '1', '3', 'RW', 'ROM/RAM', 'MULT'],
  ['2', '512', '1300', 'RW', 'ROM/RAM', 'READ'],
  ['3', '512', '1300', 'RW', 'ROM/RAM', 'READ'],
  ['4', '512', '1300', 'RW', 'ROM/RAM', 'READ']]}

The dict is keyed by port, and valued by a list of lists. Each sublist gives a memory address. The first entry is the memory index, the second is the number of words, the third is the size of each word, the fourth is the RW behavior, the 5th is the type of memory, and the last is the name of the memory module.

In [ ]:
#export
def begin_mem(q = None,
          hw='DE-SoC [USB-1]',
          dev='@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)'):
    '''
    Open a memory edit. If one is already open, pass. 
    '''
    try:
        q.begin_memory_edit(hardware_name=hw, device_name=dev)
    except:
        pass

def end_mem(q = None):
    '''
    End a memory edit. If there are none open, pass.
    '''
    try:
        q.end_memory_edit()
    except:
        pass

## Reading / writing to memory

In [ ]:
#export
@qshell
def read(q=None,
         inst=0,
         hw='DE-SoC [USB-1]',
         dev='@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)',
         begin=True,
         end=True,
         fname=None,
         delete_mif=True):
    '''Reads memory from an instance `inst` into an array `data`.
    Option `delete_mif` will delete temporary .mif file if set to `True`.'''
    if begin:
        begin_mem(q,hw=hw,dev=dev)
    fname=fname or 'read_inst{0}'.format(inst)
    q.save_content_from_memory_to_file(
        instance_index=inst,
        mem_file_path=fname,
        mem_file_type='mif')
    with open(fname, 'r') as f:
        data = mif.load(f)
        f.close()
    if delete_mif:
        os.remove(fname)
    if end:
        end_mem(q)
    return np.array(data).astype(float).astype(int)

In [ ]:
read(inst=1,hw='DE-SoC [USB-1]',dev='@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)')

array([[0, 0, 0]])

In [ ]:
#export
def format_mem(data,bits=None):
    '''
    Format input data to nested array required for memory
    '''
    if (type(data) is int) or (type(data) is float):
        data=sidis.num2ar(data,bits=bits)
    data=np.array(data).astype(int)
    if len(data.shape)==1: #if not matrix, convert for tcl
        data=np.expand_dims(data,axis=0)
    return data

In [ ]:
format_mem(0)

array([[0]])

In [ ]:
format_mem(10)

array([[1, 0, 1, 0]])

In [ ]:
format_mem([0,1,2])

array([[0, 1, 2]])

In [ ]:
#export
@qshell
def write(q=None,
          inst=0,
          data=1,
          hw='DE-SoC [USB-1]',
          dev='@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)',
          bits=None,
          begin=True,
          end=True,
          fname=None,
          delete_mif=True):
    '''Writes `data` array to memory instance `inst`.
    Option `delete_mif` will delete temporary .mif file if set to `True`.'''
    data = format_mem(data=data,bits=bits)
    fname=fname or 'write_inst{0}'.format(inst)
    if begin:
        begin_mem(q,hw,dev)
    with open(fname, 'w') as f:
        mif.dump(data, f)
        f.close()
    q.update_content_to_memory_from_file(
        instance_index=inst,
        mem_file_path=fname,
        mem_file_type='mif')
    if end:
        end_mem(q)
    if delete_mif:
        os.remove(fname)

In [ ]:
write(inst=1,data=[1,0,1])
read(inst=1)

array([[1, 0, 1]])

In [ ]:
#export
@qshell
def read_write(q=None,
              args=[[0,'w',1,1],[0,'r']],
              hw='DE-SoC [USB-1]',
              dev='@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)',
              reps=1,
              begin=True,
              end=True):
    '''
    Read/write using `args`, a list of lists/tuples each of the form
    (instance index, 'r' or 'w', data to write if 'w', bits to use if 'w').
    Can repeat operation up to `reps`.
    '''

    data=[[] for i in range(reps)]
    if begin:
        begin_mem(q=q,hw=hw,dev=dev)
    for i in range(reps):
        for tup in args:
            inst=tup[0]
            if tup[1]=='r':
                data[i]+=[read(q=q,inst=inst,hw=hw,dev=dev,begin=False,end=False)]
            elif tup[1]=='w':
                x=tup[2]
                if len(tup)==3:
                    b=None
                else:
                    b=tup[3]
                x=format_mem(data=x,bits=b)
                write(q=q,inst=inst,data=x,hw=hw,dev=dev,begin=False,end=False)
    if end:
        end_mem(q)
    if data!=[]:
        data=np.squeeze(data).astype(int)
        if np.shape(data)==():
            data=np.array([data])
        return data

In [ ]:
read_write(args=[(1,'w',0,3),(1,'r')])

array([0, 0, 0])

In [ ]:
read_write(args=[(1,'w',[1,1,1]),(1,'r')])

array([1, 1, 1])

In [ ]:
read_write(args=[(1,'w',[1,1,1]),(1,'r')],reps=2)

array([[1, 1, 1],
       [1, 1, 1]])

In [ ]:
read_write(args=[(1,'w',[1,1,1]),(1,'r'),(1,'w',0,3),(1,'r')])

array([[1, 1, 1],
       [0, 0, 0]])

In [ ]:
#export
@qshell
def read_write_all(q=None,
                 args=[[0,'w',1,1],[0,'r']],
                 fpgas=[['DE-SoC [USB-1]','@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)']],
                 reps=1,
                 begin=False,
                 end=False):
    data=[[] for i in range(len(fpgas))]
    for i,(hw,dev) in enumerate(fpgas):
        data[i]=read_write(q=q,args=args,hw=hw,dev=dev,reps=reps,begin=begin,end=end)
    if data!=[]:
        data=np.array(data).astype(int)
        return data

In [ ]:
read_write_all(fpgas=[('DE-SoC [USB-1]','@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)'),
                     ('DE-SoC [USB-2]','@2: 5CSEBA6(.|ES)/5CSEMA6/.. (0x02D020DD)')],
              args=[(1,'w',[1,1,1]),(1,'r'),(1,'w',0,3),(1,'r')])

array([[[1, 1, 1],
        [0, 0, 0]],

       [[1, 1, 1],
        [0, 0, 0]]])

## Programming, compiling, archiving

In [ ]:
#export
def program_proj(projectname,hw=None,path='output_files'):
    "Program a single FPGA using a .cdf file."
    if hw is None:
        try:
            os.system(r'quartus_pgm output_files/{0}.cdf'.format(projectname))
        except FileNotFoundError:
            print("Use Quartus Programmer GUI to generate cdf file first.")
    else:
        filename=projectname+'.cdf'
        filename=path+'\\'+filename
        hdwname='\"'+hw+'\"'
        os.system(r'quartus_pgm -c {0} {1}'.format(hdwname,filename))

In [ ]:
#export
def compile_proj(projectname):
    'Compiles `Quartus` project `projectname`.'
    os.system('quartus_sh --flow compile '+projectname)

In [ ]:
#export
def archive_proj(projectname,filename,path=''):
    "Generates a `projectname-date.qar` file which Quartus can use to restore all project files."
    os.system('quartus_sh --archive {pr} -use_file_set sr -output "{pa}{fn}.qar" {pr}'.format(pr=projectname,
                                                                                       fn=filename,
                                                                                       pa=path))

## Location and routing annotation and QSF file appending

In [ ]:
#export
def back_annotate(projectname,routing=True,logic=True):
    '''
    Writes all routing and logic to the qsf file.
    '''
    if logic:
        os.system(f'quartus_cdb {projectname} --back_annotate=lc')
    if routing:
        os.system(f'quartus_cdb {projectname} --back_annotate=routing')

In [ ]:
#export
def write_qsf(txt='',style='w',projectname='DE10',sourcename='qsf_source.qsf',set_source=False):
    '''Writes assignments `txt` to `sourcename` and includes them in `projectname`.qsf.'''
    with open(sourcename,style) as file:
        file.writelines(txt)
    if set_source: #making sure command to source is in qsf
        with open(projectname+'.qsf','r') as file:
            lines=file.readlines()
        if 'source {}'.format(sourcename) not in lines:
            with open(projectname+'.qsf','a') as file:
                file.write("\n")
                file.write('source {}'.format(sourcename))
                file.write("\n")

## LUT Masks

In [ ]:
#export
def lut_mask(lut=lambda s:1 if np.sum(s)>=2 else 0, N=4): #majority vote
    '''
    Quartus LUT mask for primitive logic cell, takes function to input and returns hex encoding
    in:
        lut (callable or list or numpy.array or dict) : function mapping over the port entry. 
        N (int) : number of ports. for standard 4-input LUT, 0=d, 1=c, 2=b, 3=a
    out:
        mask (str) : hex string encoding of the given mask
    '''
    mask=np.zeros(2**N)
    for i,s in enumerate(np.ndindex(tuple([2 for i in range(N)]))):
        if callable(lut):
            mask[i]=lut(s)
        else:
            mask[i]=lut[s]
    mask=sidis.ar2hex(mask)
    return mask

In [ ]:
lut_mask(lambda s:1 if s[0]==1 else 0)

'0x00ff'

## Timing analysis

In [ ]:
#export
def analyze_timing(projectname,
           _from = '[get_clocks {MyClock}]',
           _to = '[get_cells {MyCell}]',
           detail='summary', #or 'full_path'
           filename='timing.txt',
           npaths = 1
          ):
    '''
    Runs the timing analyzer, getting path information from `_from` to `_to`.
    Gives `detail` (summary or full_path) on `npaths` and generates `filename`.
    Returns text of that file.
    '''
    cmd=f'report_timing -from {_from} -to {_to} -setup \
        -npaths {npaths} -detail {detail} -panel_name {{Report Timing}} -multi_corner -file {filename}'
    with open(filename,'w') as f: #generate intermediary tcl file
            f.write(cmd)
    os.system(f'quartus_sta --report_script={filename} {projectname}')
    with open(filename,'r') as f: #write over intermediate file with above command then read it
            txt=f.readlines()
    return txt

In [ ]:
hide
sidis.refresh()

C:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)
